In [ ]:
# %%
import pandas as pd
from pymongo import MongoClient as mc
from datetime import datetime as dt
import win32com.client as win32
import os
os.chdir(r"path")
print(os.getcwd())
print("imported necessary libraries")
# %%
client = mc(
    "enter mongodb address")
collection = client.database.collection
print("connected to db")
# %%
prevmonth = dt.now().month-1


# %%
cursor = collection.find({"month": prevmonth})
print("created cursor")

# %%
list = list(cursor)
print("created list")
# %%
df = pd.DataFrame(list)
print("created dataframe")
# %%
result = df.groupby(['month', 'year']).agg(
    {'UniqTxn': 'nunique', 'UniqItems': 'count'})

# %%
result['AvgUniqBasketQty'] = int(round(result['UniqItems']/result['UniqTxn']))

# %%

resultnew = result.reset_index()


# %%
writer = pd.ExcelWriter(
    f'{prevmonth}-{dt.now().year}.xlsx', engine='xlsxwriter')
resultnew.to_excel(writer, sheet_name='Summary', index=False)
writer._save()
print("file created")
# writer.close()
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.Subject = f'Far East POS report for {prevmonth}-{dt.now().year}'
mail.Attachments.Add(os.getcwd()+rf"\\{prevmonth}-{dt.now().year}.xlsx")
mail.To = 'to email'
mail.CC = 'cc email;cc email'

mail.HTMLBody = rf'''<h3>Dear name,</h3>
<br>
PFA Far East POS data for {prevmonth}-{dt.now().year}.<br><br>
Regards,<br>
Fadi <br>
Data Analyst
'''
mail.send

print("file sent and successful")
# %%
